<a href="https://colab.research.google.com/github/ib565/Gemma-Fine-Tuning-API/blob/main/fine_tune_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps transformers

In [8]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

In [9]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 4096, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


In [10]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [11]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [37]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [38]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="/content/drive/MyDrive/Datasets/api_pack_python_processed.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [39]:
dataset[100]

{'conversations': [{'content': '**api_description**:Documentation of the Rest services created for CMDM.\n**lang**:Python\nYour task:\n**instruction**\nCould you kindly show me how to add a new contract record into Master Data Management (MDM) using the CMDM - Service API and its AddContract functionality?\n**output**',
   'role': 'user'},
  {'content': 'import http.client\n\nconn = http.client.HTTPConnection("13.58.88.158:8180")\n\npayload = "{\\"contract_num\\":\\"0000000610\\",\\"contract_tp\\":\\"CRED\\",\\"currency_tp\\":\\" \\",\\"frequency_tp\\":\\"\\",\\"billing_tp\\":\\"\\",\\"signed_dt\\":\\" \\",\\"executed_dt\\":\\"\\",\\"termination_dt\\":\\"\\",\\"termination_rs_tp\\":\\" \\",\\"contract_st\\":\\"Activo\\",\\"brand_id\\":\\" \\",\\"renegociation_ind\\":\\" \\",\\"renegociation_rs_tp\\":\\"\\",\\"pay_day\\":\\" \\",\\"grace_days\\":\\"\\",\\"invoice_id\\":\\"2000000827\\",\\"discount_cd\\":\\" \\",\\"next_pym_dt\\":\\" \\",\\"monthly_pym\\":\\"0.0\\",\\"monthly_pym_winss\\

In [40]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100860 [00:00<?, ? examples/s]

In [42]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [41]:
dataset[100]

{'conversations': [{'content': '**api_description**:Documentation of the Rest services created for CMDM.\n**lang**:Python\nYour task:\n**instruction**\nCould you kindly show me how to add a new contract record into Master Data Management (MDM) using the CMDM - Service API and its AddContract functionality?\n**output**',
   'role': 'user'},
  {'content': 'import http.client\n\nconn = http.client.HTTPConnection("13.58.88.158:8180")\n\npayload = "{\\"contract_num\\":\\"0000000610\\",\\"contract_tp\\":\\"CRED\\",\\"currency_tp\\":\\" \\",\\"frequency_tp\\":\\"\\",\\"billing_tp\\":\\"\\",\\"signed_dt\\":\\" \\",\\"executed_dt\\":\\"\\",\\"termination_dt\\":\\"\\",\\"termination_rs_tp\\":\\" \\",\\"contract_st\\":\\"Activo\\",\\"brand_id\\":\\" \\",\\"renegociation_ind\\":\\" \\",\\"renegociation_rs_tp\\":\\"\\",\\"pay_day\\":\\" \\",\\"grace_days\\":\\"\\",\\"invoice_id\\":\\"2000000827\\",\\"discount_cd\\":\\" \\",\\"next_pym_dt\\":\\" \\",\\"monthly_pym\\":\\"0.0\\",\\"monthly_pym_winss\\

In [44]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/100860 [00:00<?, ? examples/s]